<h1> Things to try:</h1>

- ANOVA SCORE MODELS

- DATASET WITH DIFFERENT KINDS OF OVER AND UNDER SAMPLING STRATEGY ON THE FINAL BEST MODEL WE GET WITH THE CURRENT STRATEGIES

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## Importing all the required Libraries

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time

# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections

# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Displaying all floating point numbers upto 3 decimal places
pd.options.display.float_format = '{:.3f}'.format   

In [ ]:
%%time
df = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')

In [ ]:
df.head()

In [ ]:
df.sample(20)

In [ ]:
df.describe()

In [ ]:
df.info()

<h4> It seems there are no non null values in the dataset. Let's confirm it.

In [ ]:
df.isnull().sum()

<h4>So there are no non-null values. We also know that all the data has already been scaled (because the numerical variables have undergone PCA transformation, which requires the scaling of inputs). 

<h4>More than 75% of the transactions have amount less than 100$!!

In [ ]:
## Checking the distribution of the classes in the dataset
df['Class'].value_counts() 

In [ ]:
print("The percentage of non-fraud transactions are: {}%".format(df.Class.value_counts()[0]*100/len(df)))
print("The percentage of fraud transactions are: {}%".format(df.Class.value_counts()[1]*100/len(df)))

<h4>This is an extremely skewed dataset and we will have to perform some steps to handle the highly imbalanced set of classes. Let's plot a Histogram to further visualise the extent of class imbalance.

In [ ]:
counts = df["Class"].value_counts()
plt.bar(counts.index, counts.values, color = ['g', 'r']) # Green: Non-Fraud, Red: Fraud
plt.xlabel("Class")
plt.ylabel("Value Count")
plt.title("Fraud v/s Non-Fraud Transactions")
plt.show()

<h4> Let's have a look at the mean values of all the columns for fraud as well as non fraud transactions:-

In [ ]:
print("For Fraudulent trnsactions:-")
df[df['Class'] == 1].mean().sort_values(ascending=False)

In [ ]:
print("For Non-Fraudulent trnsactions:-")
df[df['Class'] == 0].mean().sort_values(ascending=False)

* HUGE variation in the values of V1 - V28 for fraud v/s non fraud transactions. For non-fraudulent transactions, the mean values of the columns are very close to 0. For fraudulent transactions, the mean values differ drastically away from 0.
* Mean amount of fraud transaction:- 122.211
* Mean amount of non fraud transaction:- 88.291

<h4> Let's make a correlation heatmap to check the correlation between different columns

In [ ]:
corr = df.corr()
sns.heatmap(corr, cmap='RdBu', vmin=-1, vmax=1)

<h4> Due to too many properties present, we will plot the correlation with only the target variable.

In [ ]:
colors = ['#FFD700','#3B3B3C']

corr = df.corrwith(df['Class']).sort_values(ascending = False).to_frame()
corr.columns = ['Correlation']
fig,ax = plt.subplots(nrows = 1,ncols = 2,figsize = (5,10))

plt.subplot(1,2,1)
sns.heatmap(corr.iloc[:15,:],annot = True,cmap = colors,linewidths = 0.4,linecolor = 'black',cbar = False)
plt.title('Part 1')

plt.subplot(1,2,2)
sns.heatmap(corr.iloc[15:30],annot = True,cmap = colors,linewidths = 0.4,linecolor = 'black',cbar = False)
plt.title('Part 2')

fig.tight_layout(w_pad = 2)

- For feature selection, we will exclude the features having correlation values between [-0.1,0.1].
- V4, V11 are positively correlated and V7, V3, V16, V10, V12, V14, V17 are negatively correlated with the Class feature.

In [ ]:
corr_series = df.corrwith(df['Class'])
corr_series

In [ ]:
selected_series = corr_series[~corr_series.between(-0.1, 0.1)]
selected_series

In [ ]:
selected_series.index

In [ ]:
df1 = df[['V3','V4','V7','V10','V11','V12','V14','V16','V17','Class']].copy(deep = True)
df1.head()

In [ ]:
df1.describe()

<H4> Let's now resample the data.

In [ ]:
import imblearn
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [ ]:
# Data Balancing
# Defining the samplers
undersampler = RandomUnderSampler(sampling_strategy = 0.1, random_state=42)
oversampler = SMOTE(sampling_strategy=0.5, random_state=42)

# Separating input data and target values
X1 = df1.iloc[:,:9].values  # Input data
y1 = df1.iloc[:,9].values   # Target values

# Defining a pipeline
pipe = Pipeline([('undersampling', undersampler), ('oversampling', oversampler)])

# We will now use the .fit_resample() method to first fit the undersampler object to the data so that they can learn what data points to retain etc. and 
# then we will fit the oversampler object to the data to learn the patterns of the minority class and figure out the strategy of generating synthetic
# data points. We will then use the undersampler to transform the data and achieve the 0.1 sampling_strategy and then use the oversampler object to 
# transform the data to achieve it's sampling strategy.

X1, y1 = pipe.fit_resample(X1, y1)


# Counting number of values of each class after resampling
Counter(y1)

<H2> DATA BALANCE CALCULATION </H2>
    
**Sampling Strategy = (Samples of Minority class after resampling)/(Samples of Majority class after resampling)**
<br>
We are first doing undersampling and then doing oversampling

<h3>Undersampling:</h3>
Undersampling only affects the number of samples of the majority class. Therefore the number of samples of the minority class before and after undersampling remains the same.

sampling_strategy = 0.1

0.1 = (492)/(No. of Majority Class samples after undersampling)

Therefore, after undersampling: 
- No. of Minority Class Samples = 492
- No. of Majority Class Samples = 4920


<h3>Oversampling:</h3>
Oversampling only affects the number of samples of the minority class. Therefore the number of samples of the majority class before and after undersampling remains the same.

sampling_strategy = 0.5

0.5 = (No. of Minority Class samples after undersampling)/(4920)

Therefore, after undersampling: 
- No. of Minority Class Samples = 2460
- No. of Majority Class Samples = 4920

<H3>MAIN EVALUATION METRICS:- </H3>

- AUPRC Score

- Stratified K Fold Cross Validation Score

- Recall Score (Percentage of fraudulent transactions detected)
    - Even if our precision is low, it's more important for our recall to be high so that we don't classify a fraudulent transaction as non-fraud

<H3>ADDITIONAL METRICS:-</H3>

- Confusion Matrix

- ROC-AUC plot

- Model Classification Report

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import recall_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size = 0.20, random_state = 2)

In [ ]:
### In the next code cell, we will be defining a function to evaluate the model. The following functions are
### helper functions to plot confusion matrix and the ROC curve that will be used in the model evaluating function

def plot_confusion_matrix(y_test, test_preds):
    # Confusion Matrix
    cm = confusion_matrix(y_test,test_preds)
    names = ['True Neg','False Pos','False Neg','True Pos']
    counts = [value for value in cm.flatten()]
    percentages = ['{0:.2%}'.format(value) for value in cm.flatten()/np.sum(cm)]
    labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(names,counts,percentages)]
    labels = np.asarray(labels).reshape(2,2)
    sns.heatmap(cm,annot = labels,cmap = 'Blues',fmt ='')

def plot_ROC_Curve(classifier, X_test, y_test):
    plot_roc_curve(classifier, X_test, y_test)
    plt.title("ROC Curve")
    plt.show()

'''
def model_eval(classifier, X_train, y_train, X_test, y_test):
    '''
    
    '''
    You pass a defined classifier Object with the desirable hyper parameters. The function then takes the classifier
    and fits it to the training data and targets. It then calculates the predictions on the test set. Using the 
    predictions, it calculates the AUPRC score and plots the ROC curve. It also plots a confusion matrix and returns 
    the Classification Report of the model and the Recall Score. Finally, it will also calculate the Repeated 
    Stratified K Fold Cross Validation score of the model.
    
    Let's get started!
    '''
    
    '''
    
    # Fitting the model on training data
    classifier.fit(X_train, y_train)
    
    # Making predictions on test data
    test_preds = classifier.predict(X_test)
    
    # Calculating AUPRC score for the model on the test data
    auprc_score = roc_auc_score(y_test, test_preds)
    
    # Calculate Recall Score
    recall_score = recall_score(y_test, test_preds)
    
    # Calculate Repeated Stratified K Fold Cross Validation score of the model on 
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
    cross_val_score = cross_val_score(classifier, X_train, y_train, cv=cv, scoring='roc_auc').mean()
    
    
    
    print("The AUPRC score is:- ", auprc_score)
    print("The Recall score is:- ", recall_score)
    print("The Cross Validation Score is:- ", cross_val_score)
    
        
    # Plot ROC Curve
    plot_ROC_Curve(classifier, X_test, y_test)
        
    # Plot Confusion Matrix
    plot_confusion_matrix(y_test, train_preds)
    
    print(classification_report(y_test,classifier.predict(x_test)))
'''

In [ ]:
def model_eval(classifier, X_train, y_train, X_test, y_test):    
    '''
    You pass a defined classifier Object with the desirable hyper parameters. The function then takes the classifier
    and fits it to the training data and targets. It then calculates the predictions on the test set. Using the 
    predictions, it calculates the AUPRC score and plots the ROC curve. It also plots a confusion matrix and returns 
    the Classification Report of the model and the Recall Score. Finally, it will also calculate the Repeated 
    Stratified K Fold Cross Validation score of the model.
    
    Let's get started!
    '''
    
    # Fitting the model on training data
    classifier.fit(X_train, y_train)
    
    # Making predictions on test data
    test_preds = classifier.predict(X_test)

    # Calculate Repeated Stratified K Fold Cross Validation score of the model on 
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
    cross_val = cross_val_score(classifier, X_train, y_train, cv=cv, scoring='roc_auc').mean()
    
    
    
    print("The AUPRC score is:- ", '{0:.2%}'.format(roc_auc_score(y_test, test_preds)))
    print("The Recall score is:- ", '{0:.2%}'.format(recall_score(y_test, test_preds)))
    print("The Cross Validation Score is:- ", '{0:.2%}'.format(cross_val))
    
        
    # Plot ROC Curve
    plot_ROC_Curve(classifier, X_test, y_test)
        
    # Plot Confusion Matrix
    plot_confusion_matrix(y_test, test_preds)
    
    print(classification_report(y_test,test_preds))

In [ ]:
### This function will be used to just check the model performance while tuning hyperparameters only. Final evaluation
### of the model will be done by the model_eval() function
def basic_model_eval(classifier, X_train, y_train, X_test, y_test):
    classifier.fit(X_train, y_train)
    test_preds = classifier.predict(X_test)
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
    cross_val = cross_val_score(classifier, X_train, y_train, cv=cv, scoring='roc_auc').mean()
    print("The AUPRC score is:- ", '{0:.2%}'.format(roc_auc_score(y_test, test_preds)))
    print("The Recall score is:- ", '{0:.2%}'.format(recall_score(y_test, test_preds)))
    print("The Cross Validation Score is:- ", '{0:.2%}'.format(cross_val))

<H3> Now, we are ready to test different kinds of models!

 <h4>1) LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
classifier_lr = LogisticRegression(random_state = 0,C=10,penalty= 'l2')

In [ ]:
model_eval(classifier_lr, X_train, y_train, X_test, y_test)

Let's try different values of C

In [ ]:
for c in [0.5, 1, 5, 15, 20]:
    print(f"For C = {c}, the model evaluation is as follows:-")
    basic_model_eval(LogisticRegression(random_state = 0,C=c,penalty= 'l2'), X_train, y_train, X_test, y_test)
    print('\n')
    print('\n')

So C being greater than 10 isn't giving any significant benefits. We'll keep C = 10

 <h4>2) Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

for md in [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]:
    print(f"For max_depth = {md}, the model evaluation is as follows:-")
    basic_model_eval(DecisionTreeClassifier(random_state = 42,max_depth = md,min_samples_leaf = 1),
               X_train, y_train, X_test, y_test)
    print('\n')
    print('\n')

Observing that the change in recall score and AUPRC score becomes negligible after max_depth=9, we choose max_depth=9

In [ ]:
classifier_dt = DecisionTreeClassifier(random_state = 42,max_depth = 9,min_samples_leaf = 1).fit(X_train, y_train)

<H4> 4] Random Forest Classifier :

In [ ]:
from sklearn.ensemble import RandomForestClassifier
for n_est in [10, 50, 100, 200, 400, 800]:
    print(f"For n_estimators = {n_est}, the model evaluation is as follows:-")
    basic_model_eval(RandomForestClassifier(n_estimators=n_est, max_depth = 4,random_state = 42),
               X_train, y_train, X_test, y_test)
    print('\n')
    print('\n')

Highest AUPRC and Recall Score is for n_estimators=10, so we fix n_estimators=10

In [ ]:
from sklearn.ensemble import RandomForestClassifier
for md in [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]:
    print(f"For max_depth = {md}, the model evaluation is as follows:-")
    basic_model_eval(RandomForestClassifier(n_estimators=10, max_depth = md,random_state = 42),
               X_train, y_train, X_test, y_test)
    print('\n')
    print('\n')

We'll choose max_depth=11 for our final RF model

In [ ]:
classifier_rf = RandomForestClassifier(random_state=42, n_estimators=10, max_depth=11)
model_eval(classifier_rf, X_train, y_train, X_test, y_test)

<H4> 4) K-Nearest Neighbours

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
for k in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]:
    print(f"For n_neighbors = {k}, the model evaluation is as follows:-")
    basic_model_eval(KNeighborsClassifier(n_neighbors = k),
               X_train, y_train, X_test, y_test)
    print('\n')
    print('\n')

We select n_neighbors=3

In [ ]:
for l in [1, 2, 3, 4, 5, 6]:
    print(f"For p = {l}, the model evaluation is as follows:-")
    basic_model_eval(KNeighborsClassifier(n_neighbors = 3, p = l),
               X_train, y_train, X_test, y_test)
    print('\n')
    print('\n')

We select p = 1

In [ ]:
# I don't think leaf_size is being used by the model at all, so changing leaf size won't really matter anyways
# But let's just confirm it once by running the following iteration.
for leaf in range(1,7):
    print(f"For leaf_size = {leaf}, the model evaluation is as follows:-")
    basic_model_eval(KNeighborsClassifier(n_neighbors = 3, p = l, leaf_size=leaf),
               X_train, y_train, X_test, y_test)
    print('\n')
    print('\n')

So leaf doesn't play any major role.

In [ ]:
classifier_knn = KNeighborsClassifier(n_neighbors = 3, p = 1)
model_eval(classifier_knn, X_train, y_train, X_test, y_test)

- Recall Score: 98.20%
- F1 Score (for fraud): 96%
- AUPRC Score: 97.67%

In [ ]:
final_model = classifier_knn
import pickle
pickle.dump(final_model, open('final_model.pkl', 'wb'))

So
our best model is KNN with 3 neighbors and p = 1. I now want to take this model and check its performance on different kinds of oversampling and undersampling sampling strategy

In [ ]:
def get_eval_model_on_new_sampling_strategies(oversample_strategy, undersample_strategy, df=df1):
    
    ### Data Sampling
    undersampler = RandomUnderSampler(sampling_strategy=undersample_strategy, random_state=42)
    oversampler = SMOTE(sampling_strategy=oversample_strategy, random_state=42)
    X1 = df.iloc[:,:9].values  
    y1 = df.iloc[:,9].values
    pipe = Pipeline([('undersampling', undersampler), ('oversampling', oversampler)])
    X1, y1 = pipe.fit_resample(X1, y1)
    
    ### Making training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size = 0.20, random_state = 42)
    
    ### Initializing KNN Classifier Object
    curr_classifier = KNeighborsClassifier(n_neighbors = 3, p = l)
    
    ### Basic Evaluation of the model
    basic_model_eval(curr_classifier, X_train, y_train, X_test, y_test)

Let's try:-
- undersample_strategy = 0.05
- Oversample_strategy = 0.75

In [ ]:
get_eval_model_on_new_sampling_strategies(0.75, 0.05)

In [ ]:
# What happens when you put oversample_strategy=1 ? xD
get_eval_model_on_new_sampling_strategies(1, 0.05)

In [ ]:
# Oversample = 1
# Undersample = 0.01
get_eval_model_on_new_sampling_strategies(1, 0.01)

<H1>Is this causing overfitting?????????